In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import json

In [2]:
df = pd.read_csv("./data/교내 주제선택 분반_위도경도 통합_null값 반영.csv")

In [3]:
dist = pd.read_csv("./data/분반_기관_distance.csv")
cost = pd.read_csv("./data/분반_기관_cost.csv")

In [4]:
new_order = list(cost.columns[2:12]) + list(cost.columns[:2]) + list(cost.columns[12:])

In [5]:
dist = dist.reindex(columns = new_order)
cost = cost.reindex(columns = new_order)

In [6]:
cos_sim = pd.read_csv("./data/contents_similarity.csv")

In [12]:
cos_sim = cos_sim.reindex(columns = new_order)

In [175]:
cost = cost.astype(int)

In [13]:
def find_matching2(idx):
    K = int(df.loc[idx]["예산(만원)"]/2)
    N = 358
    weight = [0] + list(cost.loc[idx])
    value = [0] + list(cos_sim.loc[idx])
    dp = [[[0] * (6) for _ in range(K+1)] for _ in range(N+1)]
    choiced = [[[[]] * (6) for _ in range(K+1)] for _ in range(N+1)]
    d = [0] + list(dist.loc[idx])
    
    for i in range(1,N+1):
        for j in range(1,K+1):
            for k in range(1,6):
                if d[i] > 60:
                    dp[i][j][k] = dp[i-1][j][k]
                    choiced[i][j][k] = choiced[i-1][j][k]
                    continue
                
                if j >= weight[i] and dp[i-1][j][k] < dp[i-1][j-weight[i]][k-1] + value[i]:
                        dp[i][j][k] = dp[i-1][j-weight[i]][k-1] + value[i]
                        choiced[i][j][k] = choiced[i-1][j-weight[i]][k-1] + [i]
                else:
                    dp[i][j][k] = dp[i-1][j][k]
                    choiced[i][j][k] = choiced[i-1][j][k]
    
    for key, values in match2.items():
        tmp = []
        for v in values:
            if v < 11:
                tmp.append(v + 2)
            elif v < 13:
                tmp.append(v - 10)
            else:
                tmp.append(v)
        tmp = [i-1 for i in tmp]        
        match2[key] = tmp
    
    for i in choiced[-1][-1][::-1]:
        if(len(i) != 0):
            match2[idx] = i
            break

In [ ]:
#match2 = {}
#for i in tqdm(range(1366)):
#    find_matching2(i)

In [ ]:
"""program = pd.read_csv("./data/프로그램_통합.csv", encoding = 'cp949')
학교 = []
체험처 = []
학교_위도=[]
학교_경도=[]
체험처_위도=[]
체험처_경도=[]

for key, values in match2.items():
    #print(key)
    try:
        for i in values:
            학교.append(df['학교명'][int(key)])
            체험처.append(program['제목'][i])
            #print(df['학교명'][int(key)],":", program['제목'][i])
            학교_위도.append(df['위도'][int(key)])
            학교_경도.append(df['경도'][int(key)])
            체험처_위도.append(program['위도'][i])
            체험처_경도.append(program['경도'][i])  
    except:
        print(key, values)
matched = pd.DataFrame({'학교': 학교, '체험처': 체험처, '학교_위도':학교_위도, '학교_경도':학교_경도, '체험처_위도':체험처_위도, '체험처_경도':체험처_경도})
#matched.columns = ['학교명', '체험처']
matched"""

In [25]:
program = pd.read_csv("./data/프로그램_통합.csv", encoding = 'cp949')
maxpeople = [0] + list(program["인원"])

In [39]:
def find_matching3(idx):
    K = int(df.loc[idx]["예산(만원)"]/2)
    N = 358
    weight = list(map(int, [0] + list(cost.loc[idx])))
    value = [0] + list(cos_sim.loc[idx])
    dp = [[[0] * (6) for _ in range(K+1)] for _ in range(N+1)]
    choiced = [[[[]] * (6) for _ in range(K+1)] for _ in range(N+1)]
    d = [0] + list(dist.loc[idx])
    people = df.loc[idx]["학생수"]
    for i in range(1,N+1):
        for j in range(1,K+1):
            for k in range(1,6):
                if d[i] > 60 or maxpeople[i] < people:
                    dp[i][j][k] = dp[i-1][j][k]
                    choiced[i][j][k] = choiced[i-1][j][k]
                    continue
                #print(type(j), type(weight[i]))
                if j >= weight[i] and dp[i-1][j][k] < dp[i-1][j-weight[i]][k-1] + value[i]:
                        dp[i][j][k] = dp[i-1][j-weight[i]][k-1] + value[i]
                        choiced[i][j][k] = choiced[i-1][j-weight[i]][k-1] + [i]
                else:
                    dp[i][j][k] = dp[i-1][j][k]
                    choiced[i][j][k] = choiced[i-1][j][k]
    
    for key, values in match3.items():
        tmp = []
        for v in values:
            if v < 11:
                tmp.append(v + 2)
            elif v < 13:
                tmp.append(v - 10)
            else:
                tmp.append(v)
        tmp = [i-1 for i in tmp]        
        match3[key] = tmp
    
    for i in choiced[-1][-1][::-1]:
        if(len(i) != 0):
            match3[idx] = i
            break

In [40]:
#match3 = {}
#for i in tqdm(range(1366)):
#    find_matching3(i)

  0%|          | 0/13 [00:00<?, ?it/s]

In [42]:
program = pd.read_csv("./data/프로그램_통합.csv", encoding = 'cp949')
학교 = []
체험처 = []
학교_위도=[]
학교_경도=[]
체험처_위도=[]
체험처_경도=[]

for key, values in match3.items():
    #print(key)
    try:
        for i in values:
            학교.append(df['학교명'][int(key)])
            체험처.append(program['제목'][i])
            #print(df['학교명'][int(key)],":", program['제목'][i])
            학교_위도.append(df['위도'][int(key)])
            학교_경도.append(df['경도'][int(key)])
            체험처_위도.append(program['위도'][i])
            체험처_경도.append(program['경도'][i])  
    except:
        print(key, values)
matched = pd.DataFrame({'학교': 학교, '체험처': 체험처, '학교_위도':학교_위도, '학교_경도':학교_경도, '체험처_위도':체험처_위도, '체험처_경도':체험처_경도})
#matched.columns = ['학교명', '체험처']
matched

,학교,체험처,학교_위도,학교_경도,체험처_위도,체험처_경도
0,경북체육중학교,감성민화 그리기,35.812226,128.738053,35.982923,128.383916
1,경북체육중학교,메이커전문가 (중학생을 위한 창의융합 메이커체험_DIY 오르골만들기),35.812226,128.738053,35.992720,128.387254
2,경북체육중학교,미래식량자원인 곤충 체험,35.812226,128.738053,35.698467,128.573416
3,경북체육중학교,"코스메틱, 화장품연구원(뷰티 관련 직업군 체험)",35.812226,128.738053,36.807056,128.629346
4,경북체육중학교,쾌지나 칭칭 나네(대농놀이 체험),35.812226,128.738053,35.989578,128.399539
...,...,...,...,...,...,...
60,자인여자중학교,VR(가상현실)콘텐츠 제작자(중학생을 위한 미래 신직업 소개 - VR콘텐츠 제작),35.816043,128.827966,35.992720,128.387254
61,자인여자중학교,과학수사 KCSI / 과학수사대는 무슨 일을 할까?,35.816043,128.827966,36.807056,128.629346
62,자인여자중학교,국악기가 만들어지기까지,35.816043,128.827966,35.956530,128.487850
63,자인여자중학교,나.살.만 (나의 살만한 마을만들기) 프로젝트(메타버스),35.816043,128.827966,36.158780,128.260222


,학교,체험처,학교_위도,학교_경도,체험처_위도,체험처_경도
0,경북체육중학교,과학교육서포터즈운영,35.812226,128.738053,35.952621,128.900975
1,경북체육중학교,과학창작 VR캠프,35.812226,128.738053,35.821590,128.823560
2,경북체육중학교,"뭐야! 와인이 과학이라고!? 99.9%과일만으로 만들어지는 와인, 어떻게 만들어질까?",35.812226,128.738053,35.923620,128.896450
3,경북체육중학교,특수효과전문가(중학생을 위한 미래유망직업 소개)_토라사이언스,35.812226,128.738053,36.108491,128.187813
4,경북체육중학교,"화석과 광물의 세계(지질학자, 고고학자 체험)",35.812226,128.738053,36.158202,128.260379
...,...,...,...,...,...,...
6768,흥해중학교,건강한 부모되기(성교육),36.109218,129.341171,36.014348,129.353338
6769,흥해중학교,"검사, 검찰 수사관 체험",36.109218,129.341171,35.844450,129.217300
6770,흥해중학교,와인맛의 감정사 소믈리에 (식사와 함께하는 와인예절교육),36.109218,129.341171,35.923620,128.896450
6771,흥해중학교,쪽득한 양갱만들기(가공기능사 체험),36.109218,129.341171,36.138262,129.007378


In [181]:
matched.to_csv('IP_matched.csv', index = False)